# IMPORT

In [1]:
import pandas as pd
import numpy as np
import re


In [6]:
recipe = pd.read_json('../../personnal_data/recipes_jow.json').T.reset_index(names='recipe_name')
data_ademe = pd.read_excel("../../personnal_data/data_food.xlsx")

In [5]:
df = pd.DataFrame(columns=['unit','ingredient'])
for index, x in recipe.iterrows():
    for elt  in x.ingredients:
        if len(elt) == 1:
            elt.append(np.nan)
        if len(elt) > 2:
            elt = elt[1:]
        df.loc[len(df)] = elt

units = []
ingredients = []
for index, x in recipe.iterrows():
    for elt  in x.ingredients:
        units.append(elt[0])
        try:
            ingredients.append([elt[1]])
        except:
            pass

## weird units

In [ ]:
serie_units = pd.Series(units).astype('string')

In [ ]:
serie_units[serie_units.str.contains('|'.join(['']))]

0        80 g
1       100 g
2       1 càs
3        70 g
4           6
        ...  
7510        1
7511     15 g
7512     2 cl
7513     2 cl
7514     2 cl
Length: 7515, dtype: string

In [ ]:
#classic ones
basic_morceau_part = ['1 l', 'cl', 'ml', 'g', 'càc', 'càs', 'kg', 'pinc.']
weird_units = serie_units[~serie_units.str.contains('|'.join(basic_morceau_part))].copy()
#length ==  1: a number of object
weird_units = weird_units[weird_units.str.len() > 1].copy()
#length == 3 and contains '/': fraction of an object
weird_units = weird_units[~((weird_units.str.len() == 3) & (weird_units.str.contains('/')))].copy()


In [ ]:
weird_units.unique()

<StringArray>
[   '3 tran.', 'Facultatif',    '1 tran.',    '2 tran.',    '4 tran.',
 '1 noisette',  '1/10 bou.',   '1/5 bou.',  '1/50 bou.',       '1 cm',
       '1/10',  '3/10 bou.',  '1/2 tran.',        '1.5',     '1 sac.',
   '1/4 sac.',         '20',   '2/5 bou.',   '3/5 bou.',       '2 cm',
      '30 cm',         '15',    '8 tran.',     '1 bou.',    '6 tran.',
   '1/2 sac.',    '5 tran.',      '3/100',   '1/4 bou.',      '1 cup',
   '1/2 liv.',   '1/4 liv.',   '1/8 bou.',   '1/2 bou.',         '12',
     '2 sac.',   '1/5 sac.',   '4/5 sac.',   '2/3 bou.', '4 portions',
 '6 portions', '3 portions',    '1 boule']
Length: 43, dtype: string

- **bou.** : un bouquet -> 1/10 bou.	Ciboulette
- **tran.** : une tranche -> 3 tran.	Pancetta
- **liv.** : ???? -> 1/2 liv.	Pomme de terre
- **sac.** : un sachet -> 1 sac.	Levure chimique
- **noisette** : beurre


## weird quantity objects

In [4]:
df[df.unit.str.contains('bou.')]

,unit,ingredient
85,1/10 bou.,Ciboulette
103,1/10 bou.,Persil (frais)
124,1/10 bou.,Basilic (frais)
129,1/5 bou.,Coriandre (frais)
137,1/10 bou.,Basilic (frais)
...,...,...
3618,1/10 bou.,Coriandre (frais)
3621,1/10 bou.,Menthe (feuilles)
3639,1/10 bou.,Basilic (frais)
3642,1/10 bou.,Persil (frais)


In [8]:
if False:
    """
    Metrics to implement:
    - escalope
    - rumsteck
    - tournedos
    - steak
    - steak haché
    - paupiette
    - pavé
    - the weight (?) of unit ingredient: camembert (mean?), légumes, poissons, viandes (canard confit ? )
    - gousse
    - cuisse (Poulet (cuisse))
    - aiguillette
    - petit / gros (artichaut (gros) VS artichaut (petit))
    - cube (cube de  volaille)
    - filet 
    - feuille (algues nori)
    """
# df[(df.unit.str.len() == 1)].ingredient.unique()

voir si ça connecte à agrybalise

enlever une granularité (tout rapprocher à 'volaille'/'viande rouge')

In [13]:
agry_names = pd.Series(data_ademe['Nom du Produit en Français'].unique())
weird_names = pd.Series(df[(df.unit.str.len() == 1)].ingredient.unique())

In [21]:
agry_names[agry_names.str.contains('nori')]

55      Ao-nori (Enteromorpha sp.), séchée ou déshydratée
1077    Gracilaire ou ogonori (Gracilaria verrucosa), ...
dtype: object

In [17]:
#Names in agrybalise
weird_names[weird_names.isin(agry_names)]


38    Pain pita
dtype: object

Only one word in agrybalise : Pain pita. \
But for exemple, Ao-nori is in agrybalise and feuille (algue nori) is in the jow database.